# atmodeller

## Tutorial 3: Monte Carlo experiment

We can devise a simple Monte Carlo approach to sample the probable atmospheres that can arise for different planetary conditions.

Import the required functionality and activate the debug logger.

In [1]:
from atmodeller import debug_logger
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet, Species
from atmodeller.constraints import MassConstraint, IronWustiteBufferConstraintHirschmann, SystemConstraints
from atmodeller.interfaces import GasSpecies
from atmodeller.solubilities import PeridotiteH2O, BasaltDixonCO2, BasaltLibourelN2
from atmodeller.utilities import earth_oceans_to_kg
import numpy as np
import logging
import csv

logger = debug_logger()
logger.setLevel(logging.INFO)

In [2]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(chemical_formula='N2', solubility=BasaltLibourelN2()))
species

[09:48:06 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2O (H2O)
[09:48:06 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: H2 (H2)
[09:48:06 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: O2 (O2)
[09:48:06 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: CO (CO)
[09:48:06 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: CO2 (CO2)
[09:48:06 - atmodeller.interfaces          - INFO     ] - Creating a GasSpecies: N2 (N2)


Species([GasSpecies(chemical_formula='H2O', name_in_thermodynamic_data='H2O', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('H2O'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x12358fb90>, output=None, solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x12d84ee90>, solid_melt_distribution_coefficient=0, eos=IdealGas(critical_temperature=1, critical_pressure=1, standard_state_pressure=1)),
         GasSpecies(chemical_formula='H2', name_in_thermodynamic_data='H2', thermodynamic_class=<class 'atmodeller.interfaces.ThermodynamicDataJANAF'>, formula=Formula('H2'), thermodynamic_data=<atmodeller.interfaces.ThermodynamicDataJANAF object at 0x12365e250>, output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x10e1a5dd0>, solid_melt_distribution_coefficient=0, eos=IdealGas(critical_temperature=1, critical_pressure=1, standard_state_pressure=1)),
         GasSpecies(chemical_formula='O2', nam

In [3]:
planet: Planet = Planet()
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Mantle mass (kg) = 4208261222595110885130240.000000
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Mantle melt fraction = 1.000000
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Core mass fraction = 0.295335
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Planetary radius (m) = 6371000.000000
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Planetary mass (kg) = 5972000000000000327155712.000000
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Surface temperature (K) = 2000.000000
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Surface gravity (m/s^2) = 9.819973
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Melt Composition = None
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[09:48:06 - atmodeller.interior_atmosphere - INFO     ]

In [4]:
number_of_realisations = 10

# Parameters are normally distributed between bounds.
number_ocean_moles = np.random.uniform(1, 10, number_of_realisations)
ch_ratios = np.random.uniform(0.1, 1, number_of_realisations)
fo2_shifts = np.random.uniform(-4, 4, number_of_realisations)

# Store the output in a list.
out = []

# ppmw of Nitrogen in the mantle. 2.8 is the mantle value of N.
N_ppmw = 2.8

# The nitrogen mass is constant.
mass_N = N_ppmw * 1.0e-6 * planet.mantle_mass

for realisation in range(number_of_realisations):

    mass_H = earth_oceans_to_kg(number_ocean_moles[realisation])
    mass_C = ch_ratios[realisation] * mass_H
    constraints = SystemConstraints([
        MassConstraint(species="H", value=mass_H),
        MassConstraint(species="C", value=mass_C),
        MassConstraint(species="N", value=mass_N),
        IronWustiteBufferConstraintHirschmann(log10_shift=fo2_shifts[realisation])
    ])
    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    interior_atmosphere.solve(constraints, factor=0.1)
    out_realisation = interior_atmosphere.solution_dict

    # Include the parameters in the output.
    out_realisation['number_ocean_moles'] = number_ocean_moles[realisation]
    out_realisation['ch_ratio'] = ch_ratios[realisation]
    out_realisation['fo2_shift'] = fo2_shifts[realisation]

    out.append(out_realisation)

    filename = "atmodeller_monte_carlo_tutorial3.csv"
    print("Writing output to: %s", filename)
    fieldnames: list[str] = list(out[0].keys())
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        writer: csv.DictWriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(out)

[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Assembling constraints
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Constraints: [MassConstraint(name='mass', species='H', value=8.621920885876075e+20),
 MassConstraint(name='mass', species='C', value=2.879243280351562e+20),
 MassConstraint(name='mass', species='N', value=1.178313142326631e+19),
 IronWustiteBufferConstraintHirschmann(name='fugacity',
                                       species='O2',
                                       log10_shift=-3.8156744435576293)]
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - 3 additional (mass) constraint(s) are necessary to solve the system
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Apply fugacity constraint for O2
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:06 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:06 - atmodeller.in

Writing output to: %s atmodeller_monte_carlo_tutorial3.csv


[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:08 - at

Writing output to: %s atmodeller_monte_carlo_tutorial3.csv


[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:08 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:08 - atmodeller.interior_

...

Writing output to: %s atmodeller_monte_carlo_tutorial3.csv


[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:44 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:44 - at

Writing output to: %s atmodeller_monte_carlo_tutorial3.csv


[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 02: Setting O2 fugacity
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 00: Reaction 0: 0.5 O2 + 1.0 CO = 1.0 CO2
[09:48:45 - atmodeller.interior_atmosphere - INFO     ] - Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[09:48:45 - at

Writing output to: %s atmodeller_monte_carlo_tutorial3.csv


The simulation data is output as:

In [5]:
out

[{'H2O': 2.610581173614702,
  'H2': 205.1675843276392,
  'O2': 1.3493942200493363e-11,
  'CO': 21.600124635638622,
  'CO2': 0.06028145313179219,
  'N2': 0.3700309875324888,
  'number_ocean_moles': 5.562524085085453,
  'ch_ratio': 0.33394452564139965,
  'fo2_shift': -3.8156744435576293},
 {'H2O': 10.416291109819696,
  'H2': 0.3668822825035904,
  'O2': 6.718226958476912e-05,
  'CO': 12.06228349103546,
  'CO2': 75.11296397951124,
  'N2': 2.9400421995189445,
  'number_ocean_moles': 5.156637597715137,
  'ch_ratio': 0.2241960940533509,
  'fo2_shift': 2.8849864091560455},
 {'H2O': 7.528541225170389,
  'H2': 0.7466255763594594,
  'O2': 8.474183654557784e-06,
  'CO': 69.94590795922669,
  'CO2': 154.6925080944637,
  'N2': 2.891057800492979,
  'number_ocean_moles': 4.382656019386232,
  'ch_ratio': 0.6583740244672875,
  'fo2_shift': 1.9821196914835442},
 {'H2O': 28.27082181811663,
  'H2': 11.494859590238862,
  'O2': 5.041403487575005e-07,
  'CO': 59.59582039405704,
  'CO2': 32.14767463992171,
  'N